In [ ]:
import keras
import tensorflow
import h5py
import numpy as np
import chowdhury_s_model_builder
import os

import tensorflow as tf
from keras.api.models import Sequential, Model, load_model
from keras.api.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, LSTM, Input, GlobalAveragePooling2D
from keras.api.optimizers import Adam
from keras import regularizers
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')


In [ ]:
# Open the HDF5 file in read mode
with h5py.File("Data/Mid-Level_Perceptual_Features_Data.h5", "r") as hf:
    mid_level_train_data = hf["train"][:]  # Load the train dataset
    mid_level_label_data = hf["label"][:]  # Load the label dataset

print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
print(mid_level_label_data.shape, mid_level_label_data.dtype)

In [ ]:
label_counts = np.sum(mid_level_label_data, axis=0)

# Optional: class names if you have them
class_names = ["Melodiousness", "Articulation", "Rhythmic Complexity", "Rhythmic Stability",  "Dissonance", "Tonal Stability", "Modality"]

for i, count in enumerate(label_counts):
    print(f"{class_names[i] if i < len(class_names) else 'Class ' + str(i)}: {count}")


In [ ]:
# Open the HDF5 file in read mode
with h5py.File("Data/Emotify_Annotated_Data.h5", "r") as hf:
    emotify_train_data = hf["train"][:]  # Load the train dataset
    emotify_label_data = hf["label"][:]  # Load the label dataset

print(emotify_train_data.shape, emotify_train_data.dtype)  # Check the shape and type
print(emotify_label_data.shape, emotify_label_data.dtype)

In [ ]:
label_counts = np.sum(emotify_label_data, axis=0)

# Optional: class names if you have them
class_names = ["Melodiousness", "Articulation", "Rhythmic Complexity", "Rhythmic Stability",  "Dissonance", "Tonal Stability", "Modality"]

for i, count in enumerate(label_counts):
    print(f"{class_names[i] if i < len(class_names) else 'Class ' + str(i)}: {count}")


In [ ]:
combined_train = np.concatenate([mid_level_train_data, emotify_train_data], axis=0)
combined_label = np.concatenate([mid_level_label_data, emotify_label_data], axis=0)

In [ ]:
# Shuffle
permutation = np.random.permutation(combined_train.shape[0]) # Randomly permute the indices using "5000 + 400"
shuffled_train = combined_train[permutation]
shuffled_label = combined_label[permutation]

In [ ]:
# Print final shapes to verify
print("Shuffled Train Shape:", shuffled_train.shape)
print("Shuffled Label Shape:", shuffled_label.shape)

In [ ]:
label_counts = np.sum(shuffled_label, axis=0)

# Optional: class names if you have them
class_names = ["Melodiousness", "Articulation", "Rhythmic Complexity", "Rhythmic Stability",  "Dissonance", "Tonal Stability", "Modality"]

for i, count in enumerate(label_counts):
    print(f"{class_names[i] if i < len(class_names) else 'Class ' + str(i)}: {count}")


In [ ]:
shuffled_train = shuffled_train[..., np.newaxis]  # Add a new axis to the last dimension

In [ ]:

print("Shuffled Train Shape:", shuffled_train.shape)

In [ ]:
# mid_level_train_data = mid_level_train_data[..., np.newaxis]

# print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
# print(mid_level_label_data.shape, mid_level_label_data.dtype)

In [ ]:
# emotify_train_data = emotify_train_data[..., np.newaxis]

# print(emotify_train_data.shape, emotify_train_data.dtype)  # Check the shape and type
# print(emotify_label_data.shape, emotify_label_data.dtype)

In [ ]:
# mid_level_input_shape = mid_level_train_data.shape[1], mid_level_train_data.shape[2], mid_level_train_data.shape[3]
# emotify_input_shape = emotify_train_data.shape[1], emotify_train_data.shape[2], emotify_train_data.shape[3]
# print(mid_level_input_shape, emotify_input_shape)

In [ ]:
# with tensorflow.device('/GPU:0'):
#     input = Input(shape=(256, 1292, 1))
#     x =  Conv2D(64, (5, 5), strides=2, activation="relu", padding="valid")(input)
#     x =  BatchNormalization()(x)   
#     # 2nd Layer
#     x =  Conv2D(64, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 3rd Layer
#     x =  MaxPooling2D((2, 2))(x)
#     x =  Dropout(0.3)(x)    
#     # 4th Layer
#     x =  Conv2D(128, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 5th Layer
#     x =  Conv2D(128, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 6th Layer
#     x =  MaxPooling2D((2, 2))(x)
#     x =  Dropout(0.3)(x)    
#     # 7th Layer
#     x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 8th Layer
#     x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 9th Layer
#     x =  Conv2D(384, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 10th Layer
#     x =  Conv2D(512, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 11th Layer
#     x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x) 
#     # 12th Layer
#     # x = tfa.layers.AdaptiveAveragePooling2D(x)
#     x =  GlobalAveragePooling2D(keepdims=True)(x)
#     x =  Flatten()(x)
#     x = keras.layers.Dense(256)(x)
#     A2Mid2E_branch = keras.layers.Dense(7, activation="sigmoid")(x)
#     model = Model(inputs=input, outputs=A2Mid2E_branch, name="Mid-Level_Features")
#     model.summary()

In [ ]:
# model = load_model("model.keras")
model = load_model("fine-tuned_model.keras")

In [ ]:
for layer in model.layers:
    if layer.trainable:
        print(layer.name, layer.__class__.__name__, layer.trainable)

In [ ]:
# x = GlobalAveragePooling2D(name="InsertedGlobalAveragePooling2D")(model.layers[-5].output)
# x = Dense(256, activation="relu", name="InsertedDense2")(x)
# x = Dense(7, activation="sigmoid", name="InsertedDense1")(x)
# model = Model(inputs=model.input, outputs=x)
model.summary()

In [ ]:
for layer in model.layers[:-3]:
    layer.trainable = False
    
for layer in model.layers:
    if layer.trainable:
        print(layer.name, layer.__class__.__name__, layer.trainable)

In [ ]:
# model.compile(optimizer=Adam(learning_rate=0.0005),
model.compile(optimizer=Adam(learning_rate=3.1250e-05),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [ ]:
# model.fit(mid_level_train_data, mid_level_label_data,
model.fit(shuffled_train, shuffled_label,
                       batch_size=8,
                       epochs=5,
                       shuffle=True, verbose=1)